In [1]:
import logging
import time

import awkward as ak
from coffea import processor
from coffea.nanoevents import NanoAODSchema
from coffea.analysis_tools import PackedSelection
import copy
import hist
import matplotlib.pyplot as plt
import numpy as np
import pyhf

import mplhep as hep
hep.style.use(hep.style.ATLAS)

import utils 

In [2]:
N_FILES_MAX_PER_SAMPLE = 1
USE_DASK=False
USE_SERVICEX=False
USE_SAVED_FEATURES=False

In [3]:
fileset = utils.file_input.construct_fileset(
    N_FILES_MAX_PER_SAMPLE,
    use_xcache=False,
    af_name="coffea_casa",  
    input_from_eos=False,
    xcache_atlas_prefix=None,
)

print(f"processes in fileset: {list(fileset.keys())}")
print(f"\nexample of information in fileset:\n{{\n  'files': [{fileset['ttbar__nominal']['files'][0]}, ...],")
print(f"  'metadata': {fileset['ttbar__scaleup']['metadata']}\n}}")

processes in fileset: ['ttbar__nominal', 'ttbar__scaledown', 'ttbar__scaleup', 'ttbar__ME_var', 'ttbar__PS_var', 'single_top_s_chan__nominal', 'single_top_t_chan__nominal', 'single_top_tW__nominal', 'wjets__nominal']

example of information in fileset:
{
  'files': [https://xrootd-local.unl.edu:1094//store/user/AGC/nanoAOD/TT_TuneCUETP8M1_13TeV-powheg-pythia8/cmsopendata2015_ttbar_19980_PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1_00000_0000.root, ...],
  'metadata': {'process': 'ttbar', 'variation': 'scaleup', 'nevts': 1278695, 'xsec': 729.84}
}


In [4]:
NanoAODSchema.warn_missing_crossrefs = False # silences warnings about branches we will not use here

executor = processor.FuturesExecutor(workers=4)

run = processor.Runner(
    executor=executor,
    schema=NanoAODSchema,
    savemetrics=True,
    metadata_cache={},
    chunksize=200000)

treename = "Events"


filemeta = run.preprocess(fileset, treename=treename)  # pre-processing



Output()

In [5]:
# labels_dict = {"ttbar": 0,
#                "single_top_s_chan":1,
#               "single_top_t_chan":2,
#               "single_top_tW":3,
#               "wjets":4}

# Keep only a subset of the background for illustration
labels_dict = {"ttbar": 0,
              "single_top_t_chan":1,
              "wjets":2}

In [6]:
# function to create column accumulator from list
def col_accumulator(a):
    return processor.column_accumulator(np.array(a))

processor_base = processor.ProcessorABC
class NSBI_analysis(processor_base):
    def __init__(self):
        super().__init__()
    
    def process(self, events):
        
        # Note: This creates new objects, distinct from those in the 'events' object
        elecs = events.Electron
        muons = events.Muon
        jets = events.Jet
        
        process = events.metadata["process"]  # "ttbar" etc.
        
        if process in ["single_top_s_chan", "single_top_tW"]: 
            return {}
        
        variation = events.metadata["variation"]  # "nominal" etc.
        
        # normalization for MC
        x_sec = events.metadata["xsec"]
        nevts_total = events.metadata["nevts"]
        lumi = 3378 # /pb

        if process != "data":
            xsec_weight = x_sec * lumi / nevts_total
        else:
            xsec_weight = 1

        electron_reqs = (elecs.pt > 10) & (np.abs(elecs.eta) < 2.4) & (elecs.pt < 500)
        muon_reqs = ((muons.pt > 10) & (np.abs(muons.eta) < 2.4)) & (muons.pt < 500)
        # jet_reqs = (jets.pt > 30) & (np.abs(jets.eta) < 2.4) & (jets.isTightLeptonVeto)
        jet_reqs = (jets.pt > 15) & (np.abs(jets.eta) < 2.4)

        # Only keep objects that pass our requirements
        elecs = elecs[electron_reqs]
        muons = muons[muon_reqs]
        jets = jets[jet_reqs]

        ######### Store boolean masks with PackedSelection ##########
        selections = PackedSelection(dtype='uint64')
        
        # Inclusive selection criteria 
        selections.add("exactly_1l", (ak.num(elecs) + ak.num(muons)) == 1)
        
        selections.add("Inclusive", selections.all("exactly_1l"))

        selection = selections.all("Inclusive")
        selected_jets = jets[selection]
        selected_elecs = elecs[selection]
        selected_muons = muons[selection]
        selected_weights = np.ones(len(selected_jets)) * xsec_weight


        # grab lepton info
        leptons = ak.flatten(ak.concatenate((selected_elecs, selected_muons), axis=1), axis=-1)

        # H_T = ak.sum(selected_jets.pt, axis=-1)
        pT_lep = leptons.pt
        eta_lep = leptons.eta
        phi_lep = leptons.phi
        
        #### calculate features ####
        features = np.zeros((len(pT_lep), 3))
        
        features[:, 0] = pT_lep
        features[:, 1] = eta_lep
        features[:, 2] = phi_lep



        train_labels = np.full_like(pT_lep, labels_dict[process])


        output = {"train_labels": col_accumulator(train_labels.tolist()),
                  "weights": col_accumulator(selected_weights.tolist()),
                  "features": col_accumulator(features.tolist())}


        return output
        
    def postprocess(self, accumulator):
        return accumulator

In [ ]:
if not USE_SAVED_FEATURES:
    
    t0 = time.monotonic()
    # processing
    output, metrics = run(
        fileset,
        treename,
        processor_instance=NSBI_analysis()
    )


    exec_time = time.monotonic() - t0


    print(f"\nexecution took {exec_time:.2f} seconds")

Output()

In [ ]:
saved_data = "./cached_data/"

In [ ]:
if USE_SAVED_FEATURES:
    
    features = np.load(f"{saved_data}features.npy")
    train_labels = np.load(f"{saved_data}train_labels.npy")
    weights = np.load(f"{saved_data}weights.npy")
    
else:

    # grab features and labels and convert to np array
    features = np.array(output['features'].value)
    train_labels = np.array(output['train_labels'].value)
    weights = np.array(output['weights'].value)
    
    np.save(f"{saved_data}features.npy", features)
    np.save(f"{saved_data}train_labels.npy", train_labels)        
    np.save(f"{saved_data}weights.npy", weights)        

In [ ]:
bins=np.linspace(10.0, 500.0, num=20)
hist_lep_pt = {}

for key in labels_dict:
    
    hist_lep_pt[key], _ = np.histogram(features[:,0][train_labels==labels_dict[key]], weights = weights[train_labels==labels_dict[key]], bins=bins)

for key in labels_dict:

    hep.histplot(hist_lep_pt[key], bins=bins, label=key, density=1)

plt.yscale('log')
plt.legend(loc='upper right')

In [ ]:
bins=np.linspace(-2.5, 2.5, num=20)
hist_lep_eta = {}

for key in labels_dict:
    
    hist_lep_eta[key], _ = np.histogram(features[:,1][train_labels==labels_dict[key]], weights = weights[train_labels==labels_dict[key]], bins=bins)

for key in labels_dict:

    hep.histplot(hist_lep_eta[key], bins=bins, label=key, density=1)

plt.yscale('log')
plt.legend(loc='lower center')

In [ ]:
bins=np.linspace(-2.5, 2.5, num=20)
hist_lep_phi = {}

for key in labels_dict:
    
    hist_lep_phi[key], _ = np.histogram(features[:,2][train_labels==labels_dict[key]], weights = weights[train_labels==labels_dict[key]], bins=bins)

for key in labels_dict:

    hep.histplot(hist_lep_phi[key], bins=bins, label=key, density=1)

plt.yscale('log')
plt.legend(loc='lower center')

In [ ]:
train_labels

Training the Preselection NN
===

Now we train a multi-variate, multi-class classifier just to eliminate phase space with no signal events.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from tensorflow.keras.optimizers import Nadam


In [ ]:
# # One-hot encode labels
# encoder = OneHotEncoder(sparse_output=False, categories='auto')
# train_labels_reshaped = train_labels.reshape(-1, 1)
# train_labels_onehot = encoder.fit_transform(train_labels_reshaped)

# Standardize the input features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)  # Fit & transform training data

# Split data into training and validation sets (including weights)
X_train, X_val, y_train, y_val, weight_train, weight_val = train_test_split(
    features_scaled, train_labels, weights, test_size=0.25, random_state=42
)


# Define the neural network model
model = keras.Sequential([
    layers.Input(shape=(features.shape[1],)),  # Input layer
    layers.Dense(100, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

# Train the model with sample weights
model.fit(X_train, y_train, sample_weight=weight_train, 
          validation_data=(X_val, y_val), epochs=5, batch_size=10000)


In [ ]:
# Get predictions (softmax outputs)
pred_NN = model.predict(features_scaled)

In [ ]:

presel_score = np.log(pred_NN[:, labels_dict["ttbar"]]/(np.sum([pred_NN[:, labels_dict[key]] for key in list(labels_dict.keys())[1:]])))

print(presel_score.shape)
min_pred = np.amin(presel_score)
max_pred = np.amax(presel_score)

bins = np.linspace(min_pred, max_pred, num=150)

hist_NN_output = {}
for key in labels_dict: 
    hist_NN_output[key], _ = np.histogram(presel_score[train_labels==labels_dict[key]], weights = weights[train_labels==labels_dict[key]], bins=bins)


for key in labels_dict:  
    hep.histplot(hist_NN_output[key], bins=bins, 
             alpha=0.6, label=key, 
             density=True, linewidth=2.0)

plt.xlabel("Preselection Score", size=18)
plt.ylabel("Density", size=18)
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
np.save(f'{saved_data}preselection_score.npy', presel_score)
np.save(f'{saved_data}preselection_score.npy', presel_score)
np.save(f'{saved_data}preselection_score.npy', presel_score)

Make a selection cut for performing the NSBI analysis, balancing the number of signal events that go into the signal region (increasing sensitivity) and the feasibility of training accurate and precise NNs over a large phase space (need bigger models and more statistics).

As a first cut, we choose a loose preselection and see if it works.

In [ ]:
preselection_cut = -17.5 # Lets start with a very loose cut

for key in labels_dict:  
    hep.histplot(hist_NN_output[key], bins=bins, 
             alpha=0.6, label=key, 
             density=True, linewidth=2.0)

plt.xlabel("Preselection Score", size=18)
plt.axvline(preselection_cut, ymax=0.9, linestyle='--', label=f'preselection cut ({preselection_cut})')
plt.ylabel("Density", size=18)
plt.legend()
plt.yscale('log')
plt.show()